<a href="https://colab.research.google.com/github/sbowma15/script_fine_tune_gpt2/blob/main/script_fine_tune_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! python -m pip install transformers \
 #  pandas matplotlib numpy \
  # nltk seaborn sklearn gensim pyldavis \
 #  wordcloud textblob spacy textstat
!pip install --upgrade torch torchvision

from transformers import AutoTokenizer, AutoModelWithLMHead
import pandas as pd
import gc
import json
import torch
import itertools
import time
import datetime
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy.ma as ma

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler, random_split

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

#!pip install texthero
#!pip install -U spacy
import texthero as hero
from texthero import preprocessing
from texthero import stopwords
from sklearn.preprocessing import OneHotEncoder
from tqdm.notebook import trange, tqdm
from nltk.tokenize import sent_tokenize
import nltk, re, string, collections
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import torch
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import os
import random
import logging
import os
import pickle
import torch.nn as nn
import transformers
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import (
    GPT2Config,
    GPT2LMHeadModel,
    GPT2PreTrainedModel,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

MODEL_CLASSES = {"gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer)}

#FILE_PATH = os.path.join("script_buddy", "data", "film_text.txt")
FILE_PATH = '../content/sample_data/film_text.txt'

logger = logging.getLogger(__name__)

# Big thanks to the team at huggingface for providing their example
# on fine_tuning a dataset, this borrows heavily from that it.
# Please find a link to it below :-)
# https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py


class ScriptData(Dataset):
    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size=512,
        overwrite_cache=False,
    ):
        assert os.path.isfile(file_path)

        block_size = block_size - (
            tokenizer.model_max_length - tokenizer.max_len_single_sentence
        )

        directory, filename = os.path.split(file_path)

        # change if args are added at later point
        cached_features_file = os.path.join(
            directory, "gpt2" + "_" + str(block_size) + "_" + filename
        )

        if os.path.exists(cached_features_file) and not overwrite_cache:
            logger.info(
                f"Loading features from your cached file {cached_features_file}"
            )
            with open(cached_features_file, "rb") as cache:
                self.examples = pickle.load(cache)
                logger.debug("Loaded examples from cache")
        else:
            logger.info(f"Creating features from file {filename} at {directory}")

            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()
                logger.debug("Succesfully read text from file")

            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

            for i in range(0, len(tokenized_text) - block_size + 1, block_size):
                self.examples.append(
                    tokenizer.build_inputs_with_special_tokens(
                        tokenized_text[i : i + block_size]
                    )
                )

            logger.info(f"Saving features into cached file {cached_features_file}")
            with open(cached_features_file, "wb") as cache:

                pickle.dump(self.examples, cache, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)


if __name__ == "__main__":
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

    model.train()
    sc = ScriptData(tokenizer, file_path=FILE_PATH,overwrite_cache=True)

output_dir = "cpierse/gpt2_film_scripts"

tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = GPT2LMHeadModel.from_pretrained(output_dir)
model = model.to(device)
FILE_PATH = os.path.join("storage","data", "film_text.txt")
from torch import scriptdata
dataset = ScriptData(tokenizer= tokenizer, file_path= FILE_PATH )
script_loader = DataLoader(dataset,batch_size=4,shuffle=True)
BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 0.00002
WARMUP_STEPS = 10000
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
script_count = 0
sum_loss = 0.0
batch_count = 0

for epoch in range(EPOCHS):
    print(f"EPOCH {epoch} started" + '=' * 30)
    for idx,script in enumerate(script_loader):
        outputs = model(script.to(device), labels=script.to(device))

        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        script_count = script_count + 1
        if script_count == BATCH_SIZE:
            script_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 200:
            model.eval()
            print(f"sum loss {sum_loss}")
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 1000,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )

            print("Output:\n" + 100 * '-')
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            batch_count = 0
            sum_loss = 0.0
            model.train()

from transformers import WEIGHTS_NAME, CONFIG_NAME
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model.state_dict(), output_model_file)
model.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)
('./storage/models/vocab.json', './storage/models/merges.txt')
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
input_ids = tokenizer.encode('         He kisses her softly and takes out his gun.         ', return_tensors='pt')
model.eval()

2021-12-11 22:11:37.962 INFO    __main__: Creating features from file film_text.txt at ../content/sample_data
2021-12-11 22:11:49.304 INFO    __main__: Saving features into cached file ../content/sample_data/gpt2_512_film_text.txt


ImportError: ignored